# Ex 8b: Recommender Systems

In this exercise we will implement a recommendation system for a movies website. An interesting idea of this exercise is that it also learns by itself the features for each movie. So, we won't be designing if one features weights violence or romantic but the model will calculate it by itself.

# Problem formulation
Imagine an online movie rental website such as Netflix. Each movie ($i$) gets evaluated by each user ($j$) with a value from one to five stars. With this we can build a matrix where each value $(i, j)$ corresponds with the number of stars given by user $j$ to the movie $i$.

Furthermore, each movie can be defined by some features weighting, as an example, whether the movie has romantic or action components. With this we could define another matrix where each $(i, j)$ value corresponds to the weight of feature $j$ on the movie $i$.

Not all the users have seen all movie.s Therefore some values might not be defined. We will be creating a third array with boleean values where each $(i,j)$ will be 1 if the movie $i$ was rated by user $j$ and 0 in case not.

In [1]:
# Importing the needed libraries
# import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

# We'll use loadmap to load the matlab dataset
from scipy.io import loadmat
# Optimization module in scipy
from scipy import optimize

# tells matplotlib to embed plots within the notebook
#%matplotlib inline

In [2]:
class CollabFiltering:
    
    def __init__(self, parDef, epochs=50, lmbd=0):
        self.parDef = parDef
        self.epochs = epochs
        self.lmbd = lmbd
    
    def _randomInitialize(self, m, n):

        # We are adding 1 row to take in consideration the bias unit
        W = np.zeros((m, n))

        # The random function provides random values uniformly distributed between [0,1)
        # We'll multiply by a very small value (epsilon) to make sure the output is made
        # of very small values
        eps = 0.12

        # We get a testCases random datapoints. First we get a number of testCases indices out of m
        W = np.random.rand(m, n)*(2*eps)-eps
        return W
    
    def _RollTo1D(self, Theta):

        Theta1D = Theta[:,0].reshape(-1)
        for i in range(1, Theta.shape[1]):
            Theta1D = np.concatenate((Theta1D, Theta[:,i].reshape(-1)), axis=None)

        return Theta1D

    # Theta1D is a vector that must be enrolled into an 2D array
    # Shape is a tuple containing the sizes of the 2D array
    def _UnrollFrom1D(self, Theta1D, shape):
        # Useful variables
        m = shape[0]      # Destination shape high
        n = shape[1]      # Destination shape witdh

        Theta = np.zeros((m , n))
        Theta = np.reshape(Theta1D, (m,n), order = 'F')
        return Theta   
    
    def _CostFunction(self, parameters, parDef, R, Y, lmbd):
        
        X = self._UnrollFrom1D(parameters[:(parDef[1]*parDef[2])], (parDef[1], parDef[2]))
        Theta = self._UnrollFrom1D(parameters[(parDef[1]*parDef[2]):], (parDef[0], parDef[2]))
                               
        J = 0
        
        J = (1/2)*np.sum((np.power((X.dot(Theta.T)-Y), 2)*R)) + ((lmbd/2)*np.sum(np.power(Theta, 2))) + ((lmbd/2)*np.sum(np.power(X, 2)))
        
        X_grad = ((X.dot(Theta.T)-Y)*R).dot(Theta) + lmbd*X
        Theta_grad = ((X.dot(Theta.T)-Y)*R).T.dot(X)  + lmbd*Theta

        grad = np.concatenate((self._RollTo1D(X_grad), self._RollTo1D(Theta_grad)), axis=0)
                                   
        return J, grad
    
    def fit(self, R, Y):
        
        #Initializing parameters with both X and Theta
        X = np.zeros((parDef[1], parDef[2]))
        Theta = np.zeros((parDef[0], parDef[2]))
        
        X = self._randomInitialize(parDef[1], parDef[2])
        Theta = self._randomInitialize(parDef[0], parDef[2])
        
        initialParams = np.concatenate((self._RollTo1D(X), self._RollTo1D(Theta)), axis=0)
        
        # set options for optimize.minimize
        options = {'maxiter': self.epochs}

        # The function returns an object `OptimizeResult`
        # We use truncated Newton algorithm for optimization which is
        # equivalent to MATLAB's fminunc
        # See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy
        res = optimize.minimize(self._CostFunction,
                                initialParams,
                                (self.parDef, R, Y, self.lmbd),
                                jac=True,
                                method='TNC',
                                options=options)

        # the fun property of `OptimizeResult` object returns
        # the value of costFunction at optimized theta
        self.cost = res.fun

        # the optimized theta is in the x property
        self.params = res.x
        
        self.X = self._UnrollFrom1D(self.params[:(parDef[1]*parDef[2])], (parDef[1], parDef[2]))
        self.Theta = self._UnrollFrom1D(self.params[(parDef[1]*parDef[2]):], (parDef[0], parDef[2]))
        
        return self
        
 

In [3]:
# Apply mean normalizations to the values already rated 
def normalizeRatings(Y, R):
    Ytemp= np.where(R!=0, Y, np.nan)
    Ymean= np.nanmean(Ytemp, axis=1).reshape(Y.shape[0],1)
    Ynorm= np.nan_to_num(Ytemp)-np.where(np.isnan(Ytemp), 0, Ymean)
    return Ynorm, Ymean

def magic(n):
  n = int(n)
  if n < 3:
    raise ValueError("Size must be at least 3")
  if n % 2 == 1:
    p = np.arange(1, n+1)
    return n*np.mod(p[:, None] + p - (n+3)//2, n) + np.mod(p[:, None] + 2*p-2, n) + 1
  elif n % 4 == 0:
    J = np.mod(np.arange(1, n+1), 4) // 2
    K = J[:, None] == J
    M = np.arange(1, n*n+1, n)[:, None] + np.arange(n)
    M[K] = n*n + 1 - M[K]
  else:
    p = n//2
    M = magic(p)
    M = np.block([[M, M+2*p*p], [M+3*p*p, M+p*p]])
    i = np.arange(p)
    k = (n-2)//4
    j = np.concatenate((np.arange(k), np.arange(n-k+1, n)))
    M[np.ix_(np.concatenate((i, i+p)), j)] = M[np.ix_(np.concatenate((i+p, i)), j)]
    M[np.ix_([k, k+p], [0, k])] = M[np.ix_([k+p, k], [0, k])]
  return M 

In [23]:
# Loading all data on a dictonary
data = loadmat('ex8_movies.mat')

In [24]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

The $Y$ matrix stores the ratings $y^{(i,j)}$. 
The $R$ matrix stores the values $R{(i,j)}$, which determines whether the user $j$ gave a rating to movie $i$.

Collaborative filtering's objective is to predict movies ratings for the movies that the users have not yet rated. That is entries with $R(i,j)=0$.

The recomending system will provide the movies with the highest predicted ratings to the user.

In [25]:
Y=data['Y']
R=data['R']

In [26]:
Y.shape

(1682, 943)

In [27]:
# Average movie rating for the first movie 
print('The first movie\'s average rating is: %.4f'%np.mean(Y[0][np.where(R[0])]))

The first movie's average rating is: 3.8783


We will also work with the $X$ matrix. The $i$-row in $X$ corresponds to the feature vector $X^i$ for the $i$-th movie. The values of the vector $(x^{i}_{0}, x^{i}_{1}, x^{i}_{2},.....,x^{i}_{n})$ represent the weight of each feature on that movie. 
The $j$-row in $\theta$ corresponds to the parameter vector $\theta^j$ for the $j$-th user.

In [28]:
# Load pre-trained weights (X, Theta, num_users, num_movies, num_features)
data = loadmat('ex8_movieParams.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [38]:
# parDef[0] = num_users
# parDef[1] = num_movies
# parDef[2] = num_features
parDef = np.array([4, 5, 3])

X= data['X'][:parDef[1],:parDef[2]]
Theta = data['Theta'][:parDef[0],:parDef[2]]
Y = Y[:parDef[1],:parDef[0]]
R = R[:parDef[1],:parDef[0]]

In [39]:
cf = CollabFiltering(parDef)
params = np.concatenate((cf._RollTo1D(X), cf._RollTo1D(Theta)), axis=0)
lmbd = 0
J, grad = cf._CostFunction(params, parDef, R, Y, lmbd)

print('Cost at loaded parameters: %.2f \n(this value should be 22.22)'%J)

Cost at loaded parameters: 22.22 
(this value should be 22.22)


In [40]:
lmbd = 1.5
J, grad = cf._CostFunction(params, parDef, R, Y, lmbd)

print('Cost at loaded parameters: %.2f \n(this value should be 31.34)'%J)

Cost at loaded parameters: 31.34 
(this value should be 31.34)


### Coursera's testing scenarios

In [8]:
params = np.arange(1,15)/10
Y = magic(4)
Y = Y[:, 0:3]
R = np.array([[1, 0, 1], [1, 1, 1], [0, 0, 1], [1, 1, 0]])
parDef = np.array([3, 4, 2])
lmbd = 6

In [9]:
cf = CollabFiltering(parDef)
cf._CostFunction(params, parDef, R, Y, lmbd)

(342.07914999999997,
 array([-15.588, -22.344,  -3.359, -12.572, -18.438, -26.862,  -2.366,
        -14.744,   1.977,  -1.028,   3.186,  -5.059,  -8.26 ,  -1.342]))

## Load movies
Here we'll run the algorithm on a list of 1682 movies. We load the list of movies in <code>movList</code>.

In [10]:
# I set enconding utf8 to avoid and error recognising some characters
fhand = open('movie_ids.txt', encoding="utf8")

# This one liner replaces the following two lines of code
# for line in fhand:
#   movList.append(line.split(" ", 1)[1].replace("\n", ""))
movList=[line.split(" ", 1)[1].replace("\n", "") for line in fhand]

In [123]:
len(movList)

1682

## Set my initial ratings
Here we define an initial set of ratings for my own user. We start with a <code>my_ratings</code> array with all 0's and we set the value for rated movies. 

In [13]:
my_ratings=np.zeros((len(movList),))
my_ratings[0] = 4
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[97] = 2
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354]= 5

In [14]:
[print(f'Rated {my_ratings[i]:.0f} for {movList[i]}') for i in range(len(movList)) if my_ratings[i] > 0]

Rated 4 for Toy Story (1995)
Rated 3 for Twelve Monkeys (1995)
Rated 5 for Usual Suspects, The (1995)
Rated 4 for Outbreak (1995)
Rated 5 for Shawshank Redemption, The (1994)
Rated 3 for While You Were Sleeping (1995)
Rated 5 for Forrest Gump (1994)
Rated 2 for Silence of the Lambs, The (1991)
Rated 4 for Alien (1979)
Rated 5 for Die Hard 2 (1990)
Rated 5 for Sphere (1998)


[None, None, None, None, None, None, None, None, None, None, None]

## Learning the movie Ratings

In [78]:
# Loading all data on a dictonary
data = loadmat('ex8_movies.mat')

Y=data['Y']
R=data['R']

In [81]:
# Adding my ratings
Y = np.column_stack((my_ratings, Y))
R = np.column_stack((np.where(my_ratings !=0, 1,0), R))

# parDef[0] = num_users
parDef[0] = Y.shape[1]
# parDef[1] = num_movies
parDef[1] = Y.shape[0]
# parDef[2] = num_features
parDef[2] = 10

Ynorm, Ymean = normalizeRatings(Y, R)

In [26]:
epochs=100
lmbd = 10

cf = CollabFiltering(parDef, epochs, lmbd)
cf.fit(R, Ynorm)

In [27]:
cf.cost

39932.11132373622

In [32]:
# The Theta array contains for each user j (rows) the theta weight for each of the X parameters
cf.Theta.shape

(944, 10)

In [33]:
# The X array contains for each movie i (rows) the X parameters that "define" the movie
cf.X.shape

(1682, 10)

### Low rank matrix factorization

In [34]:
my_predictions = np.dot(cf.X, cf.Theta.T)[:, 0] + Ymean.reshape(-1)

In [37]:
# getting the indexes sorting from lower to higher
ix = np.argsort(my_predictions)


In [38]:
for i in reversed(range(len(ix)-10,len(ix))):
        print(f'Predicting {my_predictions[ix[i]]:.2f} for {movList[ix[i]]}.')

Predicting 5.00 for Santa with Muscles (1996).
Predicting 5.00 for Someone Else's America (1995).
Predicting 5.00 for Marlene Dietrich: Shadow and Light (1996) .
Predicting 5.00 for Entertaining Angels: The Dorothy Day Story (1996).
Predicting 5.00 for They Made Me a Criminal (1939).
Predicting 5.00 for Star Kid (1997).
Predicting 5.00 for Saint of Fort Washington, The (1993).
Predicting 5.00 for Prefontaine (1997).
Predicting 5.00 for Aiqing wansui (1994).
Predicting 5.00 for Great Day in Harlem, A (1994).


In [54]:
[print(f'Rated {my_ratings[i]:.0f} for {movList[i]}') for i in range(len(movList)) if my_ratings[i] > 0]

Rated 4 for Toy Story (1995)
Rated 3 for Twelve Monkeys (1995)
Rated 5 for Usual Suspects, The (1995)
Rated 4 for Outbreak (1995)
Rated 5 for Shawshank Redemption, The (1994)
Rated 3 for While You Were Sleeping (1995)
Rated 5 for Forrest Gump (1994)
Rated 2 for Silence of the Lambs, The (1991)
Rated 4 for Alien (1979)
Rated 5 for Die Hard 2 (1990)
Rated 5 for Sphere (1998)


[None, None, None, None, None, None, None, None, None, None, None]

In [83]:
[print(f'Predicting  {my_predictions[i]:.2f} for {movList[i]}') for i in range(len(movList)) if my_ratings[i] > 0]

Predicting  3.97 for Toy Story (1995)
Predicting  3.74 for Twelve Monkeys (1995)
Predicting  4.43 for Usual Suspects, The (1995)
Predicting  3.39 for Outbreak (1995)
Predicting  4.57 for Shawshank Redemption, The (1994)
Predicting  3.63 for While You Were Sleeping (1995)
Predicting  4.08 for Forrest Gump (1994)
Predicting  4.23 for Silence of the Lambs, The (1991)
Predicting  4.05 for Alien (1979)
Predicting  3.51 for Die Hard 2 (1990)
Predicting  3.03 for Sphere (1998)


[None, None, None, None, None, None, None, None, None, None, None]